In this notebook I update the training set with the results of games in the past. 

In [3]:
#import dependencies. 
from datetime import datetime
from nba_py import Scoreboard
import numpy as np
import pandas as pd
#get today's date, for the games!
"""
today = datetime.now()
day = today.day
month = today.month
year = today.year
f = open("output.txt", "a")
f.write('\n')

f.write(str(month)+'/'+str(day)+'/'+str(year)+'\n')
f.write("----------"+'\n')
f.close()
"""

def get_todays_games():
    """
    Get the game's going on today. 
    
    Parameters:
    -----------
    
    None
    
    
    Returns 
    -------
    
    matchups : arr
        array of all the matchups today. Note I skip the first element since it just denotes road, or home which is known
        now. 
        
        
    
    """
    
    
    today = datetime.now()
    day = today.day
    month = today.month
    year = today.year
 
    ex = Scoreboard(month=month,day=day,year=year)
    games = ex.game_header()['GAMECODE']
    matchups = []
    matchups.append(['road','home'])
    for game in games:
        away_team = game[-6:-3]
        home_team = game[-3:]

        if away_team == 'BKN':
            away_team = 'BRK'
        if home_team == 'BKN':
            home_team = 'BRK'
        
        matchups.append([away_team,home_team])
        
    return matchups[1:]

In [4]:
get_todays_games()

[['OKC', 'PHI'],
 ['PHX', 'CHA'],
 ['SAC', 'DET'],
 ['DAL', 'IND'],
 ['MIL', 'ORL'],
 ['BOS', 'ATL'],
 ['MEM', 'TOR'],
 ['MIA', 'CHI'],
 ['LAL', 'HOU'],
 ['CLE', 'DEN']]

In [ ]:
days_from = 

In [11]:
def create_previous_data():
    """Creates a dataset identical to the one used for the ML modeling. This is done by scraping the ngames averages
    of the teams just listed, along with the spread, and cominbing. 
    
    Returns 
    -------
    
    previous_matchups : arr
        In accordance with the designated format of how these team statistics will be shaped, I did that here. 
        For further explanation, please refer to the "relevant stats" and "splits_optimizer" repo's which explain why I 
        use certain values, this function simply puts them in that shape for the games I want to predict. 
    
    
    home_teams : arr
        Array of the home teams. Since I next have to obtain the spread of these games, I'll line them up based on the 
        name of the home team. 
    """
    today = datetime.now()
    day = today.day-2
    month = today.month
    year = today.year
    
    
    matchups = get_todays_games()
    
    #matchups
    import numpy as np
    from nba_py import team
    teams = team.TeamList()
    teamids = teams.info()
    #print('predicting matchups of ', teamids)
  #  print()
    teamids = teamids[:-15]
   # print(teamids)
    teamids = teamids[['TEAM_ID','ABBREVIATION']]
    teamids = teamids.rename(index=str, columns={"ABBREVIATION": "Team"})
    teamids = teamids.replace('BKN','BRK')
    teamids = teamids.sort_values('Team')
   # print(teamids)
    previous_matchups = []
    home_teams = []
    road_teams = []
    for matchup in matchups:
        home_teams.append(matchup[1])
        road_teams.append(matchup[0])
        game_array = []
        for team_ in matchup:
            TEAM_ID = teamids.loc[teamids['Team'] == team_].values[0,0]
         #   print(team_,TEAM_ID)
        
            TEAM_splits = team.TeamLastNGamesSplits(team_id=TEAM_ID,season='2018-19',date_to='12/19/2018')
       # print(TEAM_splits.last20())
            df = TEAM_splits.last20()
        
            #retain (and create) the columns proven to be correlated to outcome. 
            df['AST/TO'] = df['AST']/df['TOV']

            df = df[['FGM','FG3M','FTM','DREB','AST','STL',
                     'TOV','BLK','PTS','AST/TO','FG3_PCT',
                     'FG_PCT','FT_PCT']]
           # df['AST/TO'] = df['AST']/df['TOV']
            game_array.append(df.values)
        
        matchup_array = np.concatenate((game_array[0],game_array[1]),axis=1)
        previous_matchups.append(matchup_array)

    #quick formating!
    previous_matchups = np.array(previous_matchups)
    previous_matchups = previous_matchups[:,0,:]
    return previous_matchups,home_teams,road_teams

In [7]:
create_todays_data()

(array([[  43.4       ,   10.3       ,   19.        ,   34.9       ,
           22.6       ,   10.2       ,   13.8       ,    5.1       ,
          116.        ,    1.63768116,    0.35      ,    0.458     ,
            0.714     ,   42.8       ,   11.9       ,   20.3       ,
           36.2       ,   28.9       ,    8.        ,   15.6       ,
            5.4       ,  117.7       ,    1.8525641 ,    0.375     ,
            0.487     ,    0.746     ],
        [  42.        ,    9.5       ,   18.7       ,   30.1       ,
           24.3       ,   10.4       ,   14.6       ,    6.3       ,
          112.1       ,    1.66438356,    0.344     ,    0.465     ,
            0.781     ,   40.2       ,   11.5       ,   17.9       ,
           35.6       ,   23.6       ,    6.4       ,   13.8       ,
            4.8       ,  109.7       ,    1.71014493,    0.362     ,
            0.447     ,    0.773     ],
        [  42.9       ,   12.8       ,   15.1       ,   33.7       ,
           26.6       ,

In [12]:
create_todays_data()

(array([[  43.        ,   10.        ,   19.        ,   33.        ,
           22.8       ,   10.8       ,   16.2       ,    5.        ,
          115.        ,    1.40740741,    0.365     ,    0.469     ,
            0.679     ,   43.8       ,   12.2       ,   16.2       ,
           33.        ,   27.8       ,    6.5       ,   13.2       ,
            5.2       ,  116.        ,    2.10606061,    0.374     ,
            0.483     ,    0.746     ],
        [  41.8       ,    9.8       ,   18.        ,   35.8       ,
           25.        ,    9.3       ,   13.3       ,    5.8       ,
          111.3       ,    1.87969925,    0.358     ,    0.451     ,
            0.828     ,   42.3       ,   11.5       ,   14.5       ,
           36.3       ,   26.3       ,    5.3       ,   13.5       ,
            5.3       ,  110.5       ,    1.94814815,    0.354     ,
            0.456     ,    0.784     ],
        [  43.6       ,   14.        ,   19.6       ,   33.        ,
           27.        ,